In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
import cv2

from flask import Flask , request, jsonify
import base64
from PIL import Image
from io import BytesIO
from skimage import io

In [2]:
modelct = load_model('model.h5')
modelxray = load_model('covid_model.h5')

In [3]:
class_dict = {0:'COVID19',
              1:'NORMAL',
              2:'PNEUMONIA'}

In [ ]:
app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False


@app.route("/")
def hello():
    return "Hello World!"


@app.route("/predic/xray",methods=['POST']) 
def predictxray():
    data = request.json
    message = request.get_json()
    imagein = message.get('img')
    im = Image.open(BytesIO(base64.b64decode(imagein)))
    # plt.imshow(im)
    im.save('accept.png', 'png')


# file_path =  '/content/drive/MyDrive/Colab Notebooks/model_data/nonCovidTest (20).png'
    file_path= 'accept.png'
    test_image = cv2.imread(file_path)
    test_image = cv2.resize(test_image, (224,224),interpolation=cv2.INTER_NEAREST)
    # test_image = cv2.resize(file_path,(224,224))
#     plt.imshow(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    probs = modelxray.predict(test_image)
    pred_class = np.argmax(probs)

    pred_class = class_dict[pred_class]

#     print('prediction: ',pred_class)
   
    return jsonify(pred_class)

@app.route("/predic/ct",methods=['POST']) 
def predictct():
    data = request.json
    message = request.get_json()
    imagein = message.get('img')
    im = Image.open(BytesIO(base64.b64decode(imagein)))

    im.save('accept.png', 'png')

    # img = image.load_img('accept.png', grayscale=False, target_size=(64, 64))
         # img = image.load_img('drive/MyDrive/Colab Notebooks/model_data/nonCovidTest (20).png', grayscale=False, target_size=(64, 64)) 
                          # im = Image.open(BytesIO(base64.b64decode(img)))
    img = image.load_img('accept.png', grayscale=False, target_size=(64, 64))
                                 
    disease_class=['Covid-19','Non Covid-19']
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x /= 255
    custom = modelct.predict(x)
    a=custom[0]
    ind=np.argmax(a)        

    return jsonify(disease_class[ind])


if __name__ == '__main__':
    app.run() 

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Oct/2021 00:30:23] "GET / HTTP/1.1" 200 -
